In [36]:
import json
import re
import pandas as pd
from pprint import pprint
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback




file_path = Path('/Users/adrianfranz/Documents/Studium/6. Semester/Unternehmenssoftware/Projekt/data/dataset.csv')

In [37]:
dataset = pd.read_csv(file_path, sep=',')
dataset = dataset[dataset['games'] != 0]

In [38]:
# Normalization
# Splitting
# Model Selection
# Model Training
# Model Evaluation

In [39]:
# Normalization
features = ['games', 'goals', 'assists', 'yellow', 'yellowred', 'red',
       'time_played', 'age','goals_per_game', 'cards_per_game', 'assists_per_game']


In [40]:
scaler = MinMaxScaler()
dataset[features] = scaler.fit_transform(dataset[features])

In [41]:
count_2019 = len(dataset[dataset['season'] == 2017])

print("Count of rows with 2019 in 'season':", count_2019)

Count of rows with 2019 in 'season': 7751


In [42]:
X = dataset[['games', 'goals', 'assists', 'yellow', 'yellowred', 'red', 'time_played', 'age', 'goals_per_game', 'cards_per_game', 'assists_per_game']]
y = dataset['top_player']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [43]:
class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        super(F1ScoreCallback, self).__init__()
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        y_pred_val = np.round(self.model.predict(self.X_val))
        f1_val = f1_score(self.y_val, y_pred_val)
        print(f"Validation F1-score: {f1_val}")

In [44]:
model = Sequential()
model.add(Dense(12, input_dim=11, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [45]:
# Train the model with validation data
f1_callback = F1ScoreCallback(X_val, y_val)

model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_val, y_val), callbacks=[f1_callback])

Epoch 1/100
385/385 [==============================] - 3s 7ms/step
Validation F1-score: 0.0
9841/9841 [==============================] - 92s 9ms/step - loss: 0.0123 - accuracy: 0.9996 - val_loss: 0.0028 - val_accuracy: 0.9996
Epoch 2/100
385/385 [==============================] - 2s 6ms/step
Validation F1-score: 0.0
9841/9841 [==============================] - 73s 7ms/step - loss: 0.0024 - accuracy: 0.9996 - val_loss: 0.0022 - val_accuracy: 0.9996
Epoch 3/100
385/385 [==============================] - 2s 6ms/step
Validation F1-score: 0.28571428571428575
9841/9841 [==============================] - 70s 7ms/step - loss: 0.0020 - accuracy: 0.9996 - val_loss: 0.0021 - val_accuracy: 0.9996
Epoch 4/100
385/385 [==============================] - 2s 5ms/step
Validation F1-score: 0.0
9841/9841 [==============================] - 67s 7ms/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 0.0027 - val_accuracy: 0.9996
Epoch 5/100
385/385 [==============================] - 2s 4ms/step
Validation F1

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model accuracy: {accuracy}")

In [ ]:
# Make predictions for a new season data
new_season_data = np.array([X_test.iloc[0]])  # example data
prediction = model.predict(new_season_data)

print(f"Prediction: {prediction}")

In [ ]:
player_id = X_test.iloc[0]['id']
print(f"Player ID: {player_id}")